# Walidacja ostatecznego modelu grupy:
### Alicja Charuza, Mateusz Gałęziewski
###### Autorzy walidacji: Wiktor Wierzchowski, Wojciech Grabias

In [38]:
#imports
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint
from sklearn.ensemble import VotingClassifier
import joblib
import pickle

In [60]:
#load the training data
x_train = pd.read_csv('train_data/tfidf_train_train.csv', index_col=[0])
y_train = pd.read_csv('train_data/y_train_train.csv', index_col=[0])

#load the test data
x_test = pd.read_csv('train_data/tfidf_train_test.csv', index_col=[0])
y_test = pd.read_csv('train_data/y_train_test.csv', index_col=[0])

#load the validation data
x_valid = pd.read_csv('validation_data/tfidf_test.csv', index_col=[0])
y_valid = pd.read_csv('validation_data/y_test.csv', index_col=[0])

In [61]:
#solving problems with index after loading data
x_train = x_train.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)
x_valid = x_valid.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)
y_valid = y_valid.reset_index(drop=True)

In [62]:
def show_results(model, x, y):
    predict = model.predict(x)
    print('F1 score:',f1_score(y, predict))
    print('Accuracy:',accuracy_score(y, predict))
    print('Recall:',recall_score(y, predict))
    print('Precision:',precision_score(y, predict))
    print('ROC AUC:', roc_auc_score(y, model.predict_proba(x)[:, 1]))

## Regresja logistyczna

In [63]:
# no parameters
log_reg=LogisticRegression()
log_reg.fit(x_train, y_train.values.ravel())
print('Logistic Regression:')
print()
print('Train results:')
show_results(log_reg, x_train, y_train)
print()
print('Test results:')
show_results(log_reg, x_test, y_test)


Logistic Regression:

Train results:
F1 score: 0.9456044379382079
Accuracy: 0.9572043401830636
Recall: 0.9306187443130118
Precision: 0.9610806577916993
ROC AUC: 0.9906530696099143

Test results:
F1 score: 0.9309504467912266
Accuracy: 0.945886680342364
Recall: 0.9140375753278979
Precision: 0.9485010115872724
ROC AUC: 0.9856889669027739


Regresja logistyczna radzi sobie całkiem dobrze, dlatego też nie będziemy dobierać do niej hiperparametrów.

In [67]:
#WALIDACJA
print('Logistic Regression:')
print()
print('Valid results:')
show_results(log_reg, x_valid, y_valid)

Logistic Regression:

Valid results:
F1 score: 0.9328630809925684
Accuracy: 0.9471989697359948
Recall: 0.9198857284809341
Precision: 0.9462118308419574
ROC AUC: 0.9860358362989083


In [68]:
# WALIDACJA: Mimo wszystko proponujemy optymalizację hiperparametrów

# parameters = {
#     'penalty': ['l1', 'l2'],
#     'C': [0.001, 0.01, 0.1, 1, 10, 100],
#     'fit_intercept': [True, False],
#     'class_weight': [None, 'balanced'],
#     'solver': ['liblinear', 'saga']
# }

# lr = LogisticRegression()

# grid_search = GridSearchCV(lr, parameters, cv=5, n_jobs=-1)

# grid_search.fit(X_train, y_train)

# print("Best hyperparameters: ", grid_search.best_params_)
# print("Best score: ", grid_search.best_score_)

# # Evaluate best model on test data
# best_model = grid_search.best_estimator_
# print("Test score: ", test_score)

In [ ]:
#Valid results:
# F1 score: 0.939248576248572
# Accuracy: 0.9491349819541451
# Recall: 0.9198857284809341
# Precision: 0.92734718457194
# ROC AUC: 0.98919348179341

In [ ]:
# Hiperparametry rzeczywiście poprawiły wydajność pojedynczego modelu, jednak w ostatecznym soft-votingu
# okazały się pogarszać model

## Decision Tree

In [69]:
#Decision Tree
tree=DecisionTreeClassifier()  
tree.fit(x_train,y_train.values.ravel())
print('Decision Tree:')
print()
print('Train results:')
show_results(tree, x_train, y_train)
print()
print('Test results:')
show_results(tree, x_test, y_test)

Decision Tree:

Train results:
F1 score: 0.9999620853080569
Accuracy: 0.9999696914590531
Recall: 0.9999241734910524
Precision: 1.0
ROC AUC: 0.9999999980857692

Test results:
F1 score: 0.9155206286836934
Accuracy: 0.9330833981750017
Recall: 0.9085430698333924
Precision: 0.9226061915046796
ROC AUC: 0.9289625296194625


In [72]:
#WALIDACJA
print('Logistic Regression:')
print()
print('Valid results:')
show_results(tree, x_valid, y_valid)a

# Decision Tree stabilny względem zbioru walidacyjnego

Logistic Regression:

Valid results:
F1 score: 0.9126541857116024
Accuracy: 0.9309029669622071
Recall: 0.9052291640789965
Precision: 0.9202020202020202
ROC AUC: 0.926580639050538


Porównując wyniki zbioru treningowego i testowego widzimy, że drzewo się przeucza. Spróbujmy dobrać hiperparametry.

In [ ]:
param_distributions = {
    'max_depth': randint(low=1, high=20),
    'min_samples_leaf': randint(low=1, high=10),
    'min_samples_split': randint(low=2, high=10)
}

tree2 = DecisionTreeClassifier()
search = RandomizedSearchCV(tree2, param_distributions, n_iter=100, cv=5, scoring='accuracy', n_jobs=-1)
search.fit(x_train, y_train)
print(search.best_params_)

Spróbujmy teraz użyć tych parametrów do nowego modelu.

In [45]:
tree3 = DecisionTreeClassifier(max_depth=12, min_samples_leaf=1, min_samples_split=2)
tree3.fit(x_train, y_train)
print('Decision Tree with parameters:')
print()
print('Train results:')
show_results(tree3, x_train, y_train)
print()
print('Test results:')
show_results(tree3, x_test, y_test)

### WALIDACJA
# Poniższe wyniki nie mają przełożenia w rzeczywistości, są wynikiem błędnego podstawienia.
# W rzeczywistości wyniki na zbiorze testowym są marginalnie lepsze. Różnica względem zbioru walidacyjnego
# jest jednak na tyle mała, że dobranie tych hiperparametrów uznajemy za prawdiłowe

Decision Tree with parameters:

Train results:
F1 score: 0.9529038076938159
Accuracy: 0.9635994423228466
Recall: 0.9212920837124658
Precision: 0.9867619589052221
ROC AUC: 0.9798236555492125

Test results:
F1 score: 0.9172845077686802
Accuracy: 0.9364505423745604
Recall: 0.8836169419947832
Precision: 0.9536193029490616
ROC AUC: 0.9417685725243212


Dzięki wprowadzonym zmianom udało nam się uniknąć przeuczenia modelu i doprowadziliśmy do poprawienia wyników.

In [71]:
#WALIDACJA
print('Logistic Regression:')
print()
print('Valid results:')
show_results(tree3, x_valid, y_valid)

Logistic Regression:

Valid results:
F1 score: 0.9172845077686802
Accuracy: 0.9364505423745604
Recall: 0.8836169419947832
Precision: 0.9536193029490616
ROC AUC: 0.9417685725243212


In [46]:
# WALIDACJA: decyzja o przeszukaniu hiperparametrów jest w pełni uzasadniona

## Random Forest

In [47]:
rand_forest=RandomForestClassifier()
rand_forest.fit(x_train,y_train.values.ravel())
print('Random Forest:')
print()
print('Train results:')
show_results(rand_forest, x_train, y_train)
print()
print('Test results:')
show_results(rand_forest, x_test, y_test)
### WALIDACJA
# Poniższe wyniki nie mają przełożenia w rzeczywistości, są wynikiem błędnego podstawienia.
# W rzeczywistości wyniki na zbiorze testowym są marginalnie lepsze. Różnica względem zbioru walidacyjnego
# jest jednak na tyle mała, że dobranie tych hiperparametrów uznajemy za prawdiłowe

Random Forest:

Train results:
F1 score: 0.9999620853080569
Accuracy: 0.9999696914590531
Recall: 0.9999241734910524
Precision: 1.0
ROC AUC: 0.9999999942573077

Test results:
F1 score: 0.9290247377282734
Accuracy: 0.9457130120362573
Recall: 0.8909452241957521
Precision: 0.9705046678392639
ROC AUC: 0.9869592777089063


Patrząc na wyniki ponownie możemy podejrzewać, że model się przeuczył. Spróbujmy dobrać hiperparametry, aby poprawić jakość modelu. Zastosujemy do tego RandomizedSearch i crosswalidację.


In [73]:
#WALIDACJA
print('Logistic Regression:')
print()
print('Valid results:')
show_results(rand_forest, x_valid, y_valid)

Logistic Regression:

Valid results:
F1 score: 0.9290247377282734
Accuracy: 0.9457130120362573
Recall: 0.8909452241957521
Precision: 0.9705046678392639
ROC AUC: 0.9869592777089063


In [48]:
# WALIDACJA: nie widzimy żadnej crosswalidacji, należało zwiększyć przestrzeń hiperparametrów, to dało by większe szanse 
# na znalezienie konfiguracji unikającej przetrenowania

In [50]:
param_dist = {
    'max_depth': [10, 20, 30, 40, 50]
}
rf = RandomForestClassifier()
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=5,
    cv=5,
    n_jobs=-1,
    scoring='accuracy',
    random_state=42
)
random_search.fit(x_train, y_train.values.ravel())
print(random_search.best_params_)

MemoryError: Unable to allocate 144. MiB for an array with shape (2869, 6599) and data type float64

In [52]:
rand_forest_2 = RandomForestClassifier(max_depth=50)
rand_forest_2.fit(x_train, y_train.values.ravel())
print('Random Forest with parameters:')
print()
print('Train results:')
show_results(rand_forest_2, x_train, y_train)
print()
print('Test results:')
show_results(rand_forest_2, x_test, y_test)

Random Forest with parameters:

Train results:
F1 score: 0.9968051118210862
Accuracy: 0.9974540825604655
Recall: 0.9936305732484076
Precision: 1.0
ROC AUC: 0.9999886581827346

Test results:
F1 score: 0.9300762569471371
Accuracy: 0.9464064589628015
Recall: 0.893802012172401
Precision: 0.9694193722214738
ROC AUC: 0.9863331152271382


In [74]:
#WALIDACJA
print('Logistic Regression:')
print()
print('Valid results:')
show_results(rand_forest_2, x_valid, y_valid)

Logistic Regression:

Valid results:
F1 score: 0.9300762569471371
Accuracy: 0.9464064589628015
Recall: 0.893802012172401
Precision: 0.9694193722214738
ROC AUC: 0.9863331152271382


Jak widać nie zaszła większa poprawa w porównaniu do Random Forest bez hiperparametrów.

## SVM

Ze względu na długi czas przewidywania, zdecydowaliśmy się zrezygnować z użycia Support Vector Machine w ostatecznym modelu, aby przyspieszyć jego działanie w warunkach wdrożenia. Jego wyniki można zobaczyć w drugim milestonie.

## Soft Voting

In [53]:
clf = VotingClassifier(estimators=[('1', log_reg), ('2', tree3), ('3', rand_forest_2)],voting="soft")
clf.fit(x_train, y_train.values.ravel())
print('Voting:')
print()
print('Train results:')
show_results(clf, x_train, y_train.values.ravel())
print()
print('Test results:')
show_results(clf, x_test, y_test.values.ravel())

Voting:

Train results:
F1 score: 0.9688968275540714
Accuracy: 0.9757228587015822
Recall: 0.94601152562936
Precision: 0.99291683247115
ROC AUC: 0.9991194213075201

Test results:
F1 score: 0.9326291382197603
Accuracy: 0.9481896082024865
Recall: 0.8992671717799031
Precision: 0.968561872909699
ROC AUC: 0.9891303982997336


Łączac powstałe modele w VotingClassifier, uzyskaliśmy jak dotąd najlepszy rezultat na zbiorze testowym.

In [ ]:
# saving model to a file
filename = "fake_news_model.joblib"
joblib.dump(clf, filename)

In [ ]:
print('Valid results:')
show_results(clf, x_valid, y_valid.values.ravel())
# Valid results:
# F1 score: 0.92886532121
# Accuracy: 0.94219384134103
# Recall: 0.8922945872922
# Precision: 0.96856187290969
# ROC AUC: 0.9801529082095

Spójrzmy jeszcze na predykcyjność zmiennych w uzyskanym modelu.

Najpierw predykcyjność zmiennych w regreji logistycznej.

In [54]:
df_log = pd.DataFrame(log_reg.coef_.transpose(), 
             x_train.columns, 
             columns=['coef'])\
            .sort_values(by='coef', ascending=False)
print(df_log)

                 coef
reuter      26.845257
said        13.788396
tuesday      5.101296
monday       4.657351
washington   4.401255
...               ...
octob       -4.443670
novemb      -4.683527
hillari     -6.015001
video       -6.523562
via        -10.466876

[2869 rows x 1 columns]


Zobaczmy 30 zmiennych najbardziej wpływających na wynik, że news jest prawdziwy.

In [55]:
print(df_log.head(30))

                 coef
reuter      26.845257
said        13.788396
tuesday      5.101296
monday       4.657351
washington   4.401255
thursday     4.359061
that         4.279444
friday       3.896495
there        3.661521
say          3.464384
statement    3.369799
wednesday    3.351291
dont         3.347890
sunday       3.317013
what         3.219394
newslett     2.902446
unfold       2.855000
ive          2.816107
reform       2.809625
bbc          2.801923
didnt        2.751802
theyr        2.732813
london       2.731299
twitter      2.713188
he           2.667622
challeng     2.612749
year         2.581838
rule         2.576066
doesnt       2.546190
sept         2.532912


A teraz 30 zmiennych najbardziej wpływających na wynik, że news jest fałszywy.

In [56]:
print(df_log.tail(30))

                coef
post       -2.246348
cop        -2.247017
tell       -2.252402
american   -2.295933
alien      -2.331644
fbi        -2.359544
howev      -2.400975
hate       -2.491655
claim      -2.549968
obama      -2.579420
know       -2.599704
fact       -2.604193
dc         -2.611073
lie        -2.669494
even       -2.697838
breitbart  -2.773025
rep        -2.977652
entir      -3.070840
break      -3.082026
news       -3.269313
imag       -3.274048
daili      -3.371778
articl     -3.399443
go         -3.435409
wire       -4.183500
octob      -4.443670
novemb     -4.683527
hillari    -6.015001
video      -6.523562
via       -10.466876


In [ ]:
### WALIDACJA:
# Istotną częścią każdego modelu jest jego predykcyjność,
# Doceniamy uwzględnienie tych wpływających pozytywnie i negatywnie na 
# Ostateczną klasyfikację